In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout

In [2]:
# Importar el data set
dataset = pd.read_csv('atp_matches_2023.csv')
dataset.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2023-9900,United Cup,Hard,18,A,20230102,300,126203,3.0,NaN,...,62.0,47.0,15.0,12.0,9.0,9.0,9.0,3355.0,16.0,2375.0
1,2023-9900,United Cup,Hard,18,A,20230102,299,126207,NaN,NaN,...,12.0,8.0,3.0,4.0,1.0,3.0,19.0,2000.0,23.0,1865.0
2,2023-9900,United Cup,Hard,18,A,20230102,296,126203,3.0,NaN,...,62.0,51.0,7.0,12.0,2.0,2.0,9.0,3355.0,10.0,2905.0
3,2023-9900,United Cup,Hard,18,A,20230102,295,126207,NaN,NaN,...,41.0,26.0,12.0,9.0,6.0,9.0,19.0,2000.0,245.0,220.0
4,2023-9900,United Cup,Hard,18,A,20230102,292,126774,1.0,NaN,...,58.0,48.0,18.0,16.0,1.0,2.0,4.0,5550.0,16.0,2375.0


### Realizamos el tratamiento de los NaN's antes de nada

In [50]:
# Encontramos el valor más común entre 'Grass', 'Hard', y 'Clay'
most_common = dataset[dataset['surface'].isin(['Grass', 'Hard', 'Clay'])]['surface'].mode()[0]

# Reemplazamos los valores que no son 'Grass', 'Hard', o 'Clay' (los NaN's) con el valor más común
dataset.loc[~dataset['surface'].isin(['Grass', 'Hard', 'Clay']), 'surface'] = most_common

# Reemplazamos los valores NaN de la columna de puntos en el ranking de los jugadores por 0
dataset.iloc[:, [-1, -3]] = dataset.iloc[:, [-1, -3]].fillna(0)

In [51]:
'''Indice 7: ganador del partido.
Indice 15: perdedor del partido.
Indice -1: puntos en el ranking del jugador ganador
Indice -3: puntos en el ranking del jugador perdedor
Indice 2: tipo de campo del partido (grass, hard o clay)'''
X = dataset.iloc[:, [7, 15, -1, -3, 2]].values
y = dataset.iloc[:, 7].values

In [52]:
X[:, 0] = X[:, 0].astype(str)
X[:, 1] = X[:, 1].astype(str)
y = y.astype(str)

In [53]:
# Crear el codificador
encoder = OneHotEncoder(sparse_output=False)

# Ajustar el codificador con los datos de ambos jugadores y 'y'
encoder.fit(X[:, [0, 1]].reshape(-1, 1))

# Transformar las columnas de los jugadores
player1_onehot = encoder.transform(X[:, 0].reshape(-1, 1))
player2_onehot = encoder.transform(X[:, 1].reshape(-1, 1))

# Transformar el vector objetivo 'y' con el mismo codificador 'X'
y = encoder.transform(y.reshape(-1, 1))

# Concatenar las columnas one-hot con el resto de tus datos
X = np.concatenate([player1_onehot, player2_onehot, X[:, 2:]], axis=1)


In [54]:
# Matriz de booleanos aleatorios del mismo tamaño que tus datos
swap = np.random.rand(len(X)) > 0.5

# Intercambiamos las posiciones de los jugadores en 'X' donde 'swap' es True
X[swap, :440], X[swap, 440:880] = X[swap, 440:880], X[swap, :440].copy()

# Intercambiamos las posiciones de los puntos en el ranking donde 'swap' es True
X[swap, 880], X[swap, 881] = X[swap, 881], X[swap, 880].copy()

In [55]:
# Creamos el escalador
scaler = MinMaxScaler()

# Ajustamos y transformamos las dos últimas columnas de tus datos
X[:, [880, 881]] = scaler.fit_transform(X[:, [880, 881]])

In [56]:
# Esto es para el tipo de campo
# Crear el codificador
encoder2 = OneHotEncoder(sparse_output=False)

# Ajustar y transformar la columna del tipo de campo
field_type_onehot = encoder2.fit_transform(X[:, 882].reshape(-1, 1))

# Concatenar la columna one-hot con el resto de los datos
X = np.concatenate([X[:, :882], field_type_onehot, X[:, 883:]], axis=1)

In [57]:
from sklearn.model_selection import train_test_split

# Luego, dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [58]:
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')

In [59]:
# Definimos el modelo
model = Sequential()

# Añadir la capa de entrada y la primera capa oculta
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# Añadir la segunda capa oculta
# model.add(Dense(units=32, activation='tanh'))

# Añadir la capa de salida
model.add(Dense(units=y.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
75/75 [==============================] - 1s 2ms/step - loss: 5.9129 - accuracy: 0.0423
Epoch 2/30
75/75 [==============================] - 0s 2ms/step - loss: 5.0319 - accuracy: 0.0628
Epoch 3/30
75/75 [==============================] - 0s 2ms/step - loss: 4.6464 - accuracy: 0.1143
Epoch 4/30
75/75 [==============================] - 0s 2ms/step - loss: 4.4118 - accuracy: 0.1750
Epoch 5/30
75/75 [==============================] - 0s 2ms/step - loss: 4.1429 - accuracy: 0.2877
Epoch 6/30
75/75 [==============================] - 0s 2ms/step - loss: 3.8349 - accuracy: 0.3882
Epoch 7/30
75/75 [==============================] - 0s 2ms/step - loss: 3.5013 - accuracy: 0.4644
Epoch 8/30
75/75 [==============================] - 0s 2ms/step - loss: 3.1561 - accuracy: 0.5209
Epoch 9/30
75/75 [==============================] - 0s 2ms/step - loss: 2.8164 - accuracy: 0.5833
Epoch 10/30
75/75 [==============================] - 0s 2ms/step - loss: 2.4963 - accuracy: 0.6219
Epoch 11/30
75/75 [

In [60]:
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [70]:
# Hacemos la predicción
prediccion_test = model.predict(X_test)
# La predicción es un array con las probabilidades de cada clase
# Podemos obtener la clase con la probabilidad más alta con argmax
clase_predicha_test = np.argmax(prediccion_test, axis=1)

# Convertimos clase_predicha_test en un array 2D
clase_predicha_test_2D = np.eye(encoder.categories_[0].shape[0])[clase_predicha_test]

# Usamos el codificador para transformar la clase predicha a su etiqueta original
etiqueta_predicha_test = encoder.inverse_transform(clase_predicha_test_2D)

print(etiqueta_predicha_test)

19/19 [==============================] - 0s 1ms/step
[['100644']
 ['202104']
 ['106423']
 ['106043']
 ['106331']
 ['105173']
 ['105916']
 ['126774']
 ['111581']
 ['106432']
 ['111153']
 ['121411']
 ['208103']
 ['127157']
 ['200670']
 ['206173']
 ['207518']
 ['105173']
 ['132686']
 ['105526']
 ['200175']
 ['106065']
 ['106421']
 ['210097']
 ['126774']
 ['104792']
 ['106281']
 ['134770']
 ['136440']
 ['207989']
 ['105777']
 ['111153']
 ['202103']
 ['105916']
 ['209098']
 ['207686']
 ['200325']
 ['122330']
 ['111513']
 ['111797']
 ['126239']
 ['200005']
 ['200325']
 ['105870']
 ['106233']
 ['126774']
 ['104926']
 ['202103']
 ['126207']
 ['111513']
 ['106233']
 ['202103']
 ['210097']
 ['105870']
 ['200005']
 ['208103']
 ['111153']
 ['127157']
 ['200325']
 ['104755']
 ['200221']
 ['208103']
 ['122298']
 ['106218']
 ['126774']
 ['111797']
 ['126239']
 ['121411']
 ['106421']
 ['132283']
 ['133835']
 ['106421']
 ['105526']
 ['144684']
 ['126094']
 ['105173']
 ['208014']
 ['126094']
 ['126205']

In [79]:
'''Código para corregir el error de que haga predicciones con jugadores que no jugaron el partido'''
# Hacemos la predicción
predicciones = []
for i in range(len(X_test)):
    # Asegúrate de que estás seleccionando la i-ésima fila de X_test
    x_test_i = X_test[i, :]
    prediccion_test = model.predict(np.array([x_test_i]))
    # La predicción es un array con las probabilidades de cada clase
    # Podemos obtener la clase con la probabilidad más alta con argmax
    clase_predicha_test = np.argmax(prediccion_test)

    # Convertimos clase_predicha_test en un array 2D
    clase_predicha_test_2D = np.eye(encoder.categories_[0].shape[0])[clase_predicha_test]

    # Reshape clase_predicha_test_2D to be 2D
    clase_predicha_test_2D = clase_predicha_test_2D.reshape(1, -1)

    # Usamos el codificador para transformar la clase predicha a su etiqueta original
    etiqueta_predicha_test = encoder.inverse_transform(clase_predicha_test_2D)

    # Añadimos la etiqueta predicha a la lista de predicciones
    predicciones.append(etiqueta_predicha_test)

# Imprimimos todas las predicciones
print(predicciones)

1/1 [==============================] - 0s 16ms/step
[array([['100644']], dtype=object), array([['202104']], dtype=object), array([['106423']], dtype=object), array([['106043']], dtype=object), array([['106331']], dtype=object), array([['105173']], dtype=object), array([['105916']], dtype=object), array([['126774']], dtype=object), array([['111581']], dtype=object), array([['106432']], dtype=object), array([['111153']], dtype=object), array([['121411']], dtype=object), array([['208103']], dtype=object), array([['127157']], dtype=object), array([['200670']], dtype=object), array([['206173']], dtype=object), array([['207518']], dtype=object), array([['105173']], dtype=object), array([['132686']], dtype=object), array([['105526']], dtype=object), array([['200175']], dtype=object), array([['106065']], dtype=object), array([['106421']], dtype=object), array([['210097']], dtype=object), array([['126774']], dtype=object), array([['104792']], dtype=object), array([['106281']], dtype=object), ar

In [62]:
# Usamos el codificador para transformar las etiquetas codificadas a sus etiquetas originales
etiqueta_y_test = encoder.inverse_transform(y_test)

print(etiqueta_y_test)

[['100644']
 ['126205']
 ['106423']
 ['106043']
 ['106331']
 ['105173']
 ['105916']
 ['105526']
 ['111581']
 ['144684']
 ['200303']
 ['117365']
 ['208103']
 ['111190']
 ['200670']
 ['106421']
 ['111797']
 ['105173']
 ['132686']
 ['105526']
 ['200175']
 ['106065']
 ['106421']
 ['210097']
 ['126774']
 ['106423']
 ['106281']
 ['134770']
 ['106214']
 ['207989']
 ['200282']
 ['106214']
 ['126523']
 ['105916']
 ['209098']
 ['207686']
 ['133975']
 ['122330']
 ['111513']
 ['111797']
 ['208300']
 ['200005']
 ['200325']
 ['105870']
 ['106233']
 ['126774']
 ['106065']
 ['202103']
 ['126207']
 ['111513']
 ['106233']
 ['202103']
 ['210097']
 ['111190']
 ['105173']
 ['104918']
 ['104755']
 ['127157']
 ['106331']
 ['106000']
 ['200221']
 ['105138']
 ['122298']
 ['126239']
 ['126774']
 ['210097']
 ['133297']
 ['121411']
 ['106421']
 ['132283']
 ['133835']
 ['106421']
 ['105526']
 ['111460']
 ['126094']
 ['105173']
 ['126845']
 ['126094']
 ['126205']
 ['106281']
 ['111815']
 ['202104']
 ['111815']
 ['1

In [85]:
# Dividimos X_test en dos partes
X_test_1 = X_test[:, :440]
X_test_2 = X_test[:, 440:880]

# Aplicamos inverse_transform a cada parte
jugadores_test_1 = encoder.inverse_transform(X_test_1).ravel()
jugadores_test_2 = encoder.inverse_transform(X_test_2).ravel()

# Verificamos si cada etiqueta predicha está en los jugadores en juego
ganadores_en_juego_1 = np.isin(predicciones, jugadores_test_1)
ganadores_en_juego_2 = np.isin(predicciones, jugadores_test_2)

# Imprimimos los casos donde el ganador predicho no está en el juego
print("Ganadores predichos que no están en el juego:", np.array(predicciones)[~ganadores_en_juego_1], np.array(predicciones)[~ganadores_en_juego_2])


Ganadores predichos que no están en el juego: ['121411' '121411' '144684' '207678' '105732' '131951' '105807' '105208'
 '209080' '144684' '126952' '111576' '106220' '208134' '208182' '200270'] ['109739' '208278' '200059' '200059' '105807' '144707' '105208' '200384'
 '105902' '109686' '205734' '106210' '144707' '205734' '208182' '202262']


In [83]:
# Veamos cuántos partidos ha acertado para el test
aciertos = 0
for i in range(len(predicciones)):
    if predicciones[i] == etiqueta_y_test[i]:
        aciertos += 1
print(f"Precisión del test: {np.round(aciertos / len(predicciones)*100, 2)}%")

Precisión del test: 63.38%
